Make this notebook as wide as the browser window allows:

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Imports:

In [2]:
from dashboard import make_dashboard
from data_manager import DataManager

List all the files to load, then put them in the data manager

In [5]:
files = ['SPE10.mpk']
dm = DataManager.build(files)

make_dashboard(dm)